# RAGEx for RAG

basierend auf: https://dl.acm.org/doi/pdf/10.1145/3626772.3657660

In [1]:
import sys
from pathlib import Path

# Füge das Projektverzeichnis (mit `src/`) dem Python-Pfad hinzu, egal von wo das Notebook gestartet wird.
project_root = next((p for p in [Path.cwd()] + list(Path.cwd().parents) if (p / 'src').exists()), None)
if project_root is None:
    raise RuntimeError("\"src\"-Verzeichnis nicht gefunden. Bitte Notebook im Projekt laufen lassen.")
root_str = str(project_root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)


In [3]:
from src.modules.explainers.rag_ex_explainable import RAGExExplainable, RAGExConfig
from src.modules.rag.rag_engine import RAGEngine
from src.modules.rag.multihop_rag_engine import MultiHopRAGEngine, _format_documents
from src.modules.llm.llm_client import LLMClient
from src.modules.loader.medmcqa_data_loader import MedMCQADataLoader, format_medmcqa_question
from src.modules.loader.statspearls_data_loader import StatPearlsDataLoader
from src.evaluation.evaluator import Evaluator

import tomllib

/home/nilspoethkow/Code/Uni/XAI/xai-rag/.venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Real data example

In [4]:
config_path = project_root / "config.toml"
config = {}

if config_path.exists():
    with open(config_path, "rb") as f:
        config = tomllib.load(f)

medmcqa_config = config.get("medmcqa") or {}
rag_config = config.get("rag") or {}
llm_config = config.get("llm") or {}

llm_model = llm_config.get("model", "gemma3:4b")
llm_provider = llm_config.get("provider", "ollama")

client = LLMClient(provider=llm_provider, model_name=llm_model)

SPLIT = medmcqa_config.get("split", "val")
PERSIST_DIR = project_root / "data" / "vector_db_statpearls"
NUM_HOPS = rag_config.get('n_hops', 2)

In [5]:
stat_loader = StatPearlsDataLoader(root_dir=str(project_root / "data"))
documents, stats = stat_loader.setup()

rag_engine = RAGEngine(persist_dir=str(PERSIST_DIR))
rag_engine.setup(documents=documents)

multi_hop = MultiHopRAGEngine(rag_engine=rag_engine, llm_client=client, num_hops=NUM_HOPS)
evaluator = Evaluator()

Loading existing vector store from /home/nilspoethkow/Code/Uni/XAI/xai-rag/data/vector_db_statpearls...
RagEngine ready.
Connecting to local Ollama (gemma3:4b)...


In [ ]:
med_loader = MedMCQADataLoader()
questions = med_loader.setup(split=SPLIT, as_documents=False, limit=1)

if not questions:
    raise RuntimeError("No MedMCQA questions loaded.")

results = []
for item in questions:
    question_text = format_medmcqa_question(item)
    if not question_text:
        continue

    trace, _ = multi_hop.run_and_trace(question_text, extra='Only answer based on your context not your knowledge. Do not include any explanations, reasoning, or extra fields.\n Example: Final Answer: B: Housing')
    final_answer = (trace.get("final_answer") or "").strip()

    context_blocks = []
    for hop in trace.get("hops", []):
        docs = hop.get("documents_passed_to_context")
        if docs is None:
            continue
        
        for doc in docs:
            content = getattr(doc, "page_content", None)
            if content is None:
                content = str(doc)
            
            context_blocks.append(str(content).strip())
            context_blocks.append("\n")

    context = "\n\n".join([c for c in context_blocks if c])
    
    config = RAGExConfig()
    config.pertubation_depth = 1
    config.pertubation_mode = 'sentences'
    explainer = RAGExExplainable(llm_client=client, config=config)
    explanation = explainer.explain(query=question_text, answer=final_answer, context=context)
    metrics = explainer.metrics()

    perturbed_answers = []
    for result_item in explanation.get("results", []):
        for detail in result_item.get("details", []):
            perturbed_answer = detail.get("perturbed_answer")
            if perturbed_answer:
                perturbed_answers.append(perturbed_answer)

    answer_scores = evaluator.evaluate(perturbed_answers, baseline_answer=final_answer)

    feature_scores = sorted(
        (
            (result_item.get("segment", ""), result_item.get("token", ""), result_item.get("importance", 0.0))
            for result_item in explanation.get("results", [])
        ),
        key=lambda x: x[2],
        reverse=True,
    )

    results.append(
        {
            "question": question_text,
            "final_answer": final_answer,
            "trace": trace,
            "explanation": explanation,
            "metrics": metrics,
            "answer_scores": answer_scores,
            "feature_scores": feature_scores,
        }
    )

print(results[0]["question"])
print(results[0]["final_answer"])
result = results[0]["explanation"]
metrics = results[0]["metrics"]
answer_scores = results[0]["answer_scores"]
feature_scores = results[0]["feature_scores"]


--- Starting Multi-Hop Search for: 'Tensor veli palatini is supplied by:

Options:
A: Facial nerve
B: Trigeminal nerve
C: Glossopharyngeal nerve
D: Pharyngeal plexus' ---

[ Hop 1 ]
Executing search with query: 'Tensor veli palatini is supplied by:

Options:
A: Facial nerve
B: Trigeminal nerve
C: Glossopharyngeal nerve
D: Pharyngeal plexus'
Generating next query...

[ Hop 2 ]
Executing search with query: '“Tensor veli palatini innervation”'

Generating final answer...

--- Multi-Hop Search Complete. Final Answer: A: Facial nerve ---
--- Multi-Hop Context: 

 ('<doc id="chunk-1-1" from_hop="1" search_query="Tensor veli palatini is supplied by:\n\nOptions:\nA: Facial nerve\nB: Trigeminal nerve\nC: Glossopharyngeal nerve\nD: Pharyngeal plexus">\n[Document(id=\'8c3fde08-f0f6-4e76-b43d-0f2eb9e732c3\', metadata={\'title\': \'Essential organization of the sympathetic nervous system.\', \'chunk_id\': \'e5f3f1e295ede7fb096b0093e50b1ad13c0b0a3d0f6b6736d16f0681199ae9a1\', \'chunk_index\': 14, \'s

In [8]:
print("Explain metrics:", metrics)
print("Answer scores:", answer_scores)
print("Top feature scores:")
for segment, token, score in feature_scores[:10]:
    token_display = str(token).replace("\n", "\\n")
    print(f"{segment}\t{token_display}\t{score:.3f}")
print("----------------------------------------")
print(RAGExExplainable.prettify(result))
print(results[0]["trace"]["final_answer"])

Explain metrics: {'duration_seconds': 3368.9985232499894, 'steps': 79}
Answer scores: {'accuracy': 0.1999999999999997, 'f1': 0.33333333333333304, 'mean_jaccard': 0.1999999999999997, 'details': [{'answer': 'B: Trigeminal nerve', 'jaccard': 0.2, 'f1': 0.3333333333333333}, {'answer': 'B: Trigeminal nerve', 'jaccard': 0.2, 'f1': 0.3333333333333333}, {'answer': 'B: Trigeminal nerve', 'jaccard': 0.2, 'f1': 0.3333333333333333}, {'answer': 'B: Trigeminal nerve', 'jaccard': 0.2, 'f1': 0.3333333333333333}, {'answer': 'B: Trigeminal nerve', 'jaccard': 0.2, 'f1': 0.3333333333333333}, {'answer': 'B: Trigeminal nerve', 'jaccard': 0.2, 'f1': 0.3333333333333333}, {'answer': 'B: Trigeminal nerve', 'jaccard': 0.2, 'f1': 0.3333333333333333}, {'answer': 'B: Trigeminal nerve', 'jaccard': 0.2, 'f1': 0.3333333333333333}, {'answer': 'B: Trigeminal nerve', 'jaccard': 0.2, 'f1': 0.3333333333333333}, {'answer': 'B: Trigeminal nerve', 'jaccard': 0.2, 'f1': 0.3333333333333333}, {'answer': 'B: Trigeminal nerve', 'j

Lesson learned:

Bei langen Text brauchen wir eine größere Pertubation damit überhaupt ein Impact sichtbar ist.

### Minimal Test

In [7]:
explainer = RAGExExplainable(llm_client=client)

# Dein einfaches Beispiel
question = "Welche Autofarbe hat Tom?"
context = "Max hat ein rotes Auto. Tom ein blaues"

# Baseline Answer generieren
baseline_answer = "blau"  # oder du lässt das LLM entscheiden

# Erklärung
explanation = explainer.explain(query=question, answer=baseline_answer, context=context)

# Output
print("Baseline Answer:", baseline_answer)
print("\nTop Features:")
for item in sorted(explanation["results"], key=lambda x: x["importance"], reverse=True)[:5]:
    print(f"{item['token']:15s} → importance: {item['importance']:.3f}")

print("\n" + "="*60)
print(RAGExExplainable.prettify(explanation))

Perturbating 1 of 8
Perturbating 2 of 8
Perturbating 3 of 8
Perturbating 4 of 8
Perturbating 5 of 8
Perturbating 6 of 8
Perturbating 7 of 8
Perturbating 8 of 8
Baseline Answer: blau

Top Features:
blaues          → importance: 1.000
ein             → importance: 0.823
Max             → importance: 0.222
hat             → importance: 0.222
ein             → importance: 0.222

segment	token	importance	strategies
context	Max	0.222	leave_one_out:raw=0.19880123,sim=0.80119877 | random_noise:raw=0.19880123,sim=0.80119877
context	hat	0.222	leave_one_out:raw=0.19880123,sim=0.80119877 | random_noise:raw=0.19880123,sim=0.80119877
context	ein	0.222	leave_one_out:raw=0.19880123,sim=0.80119877 | random_noise:raw=0.19880123,sim=0.80119877
context	rotes	0.222	leave_one_out:raw=0.19880123,sim=0.80119877 | random_noise:raw=0.19880123,sim=0.80119877
context	Auto.	0.222	leave_one_out:raw=0.19880123,sim=0.80119877 | random_noise:raw=0.19880123,sim=0.80119877
context	Tom	0.222	leave_one_out:raw=0.19880123,

# Simple German QA Dataset

Documents are created for each sentence. 

In [7]:
from src.evaluation.simple_qa_dataset import get_all_contexts_as_documents, get_dataset

dataset = get_dataset()
all_documents = get_all_contexts_as_documents()

config = RAGExConfig()
config.pertubation_depth = 1
config.pertubation_mode = 'sentences'
explainer = RAGExExplainable(llm_client=client, config=config)

results = []
for i, item in enumerate(dataset):
    documents = all_documents[i][1]

    evidence = item["evidence"]
    question_text = item["question"]
    answer_prompt = client._create_final_answer_prompt(question_text, _format_documents(documents, from_query=question_text), extra='Only answer based on your context not your knowledge. Do not include any explanations, reasoning, or extra fields.\n Example: Final Answer: Housing')
    final_answer_response = client._base_llm.invoke(answer_prompt)
    final_answer = final_answer_response.content.strip()

    gt = item["answer"]

    context_blocks = []
    for doc in documents:
        content = getattr(doc, "page_content", None)
        if content is None:
            content = str(doc)
        
        context_blocks.append(str(content).strip())
        context_blocks.append("\n")

    context = "\n\n".join([c for c in context_blocks if c])

    explanation = explainer.explain(query=question_text, answer=final_answer, context=context, ground_truth_evidence=evidence)
    metrics = explainer.metrics()
    perturbed_answers = []
    for result_item in explanation.get("results", []):
        for detail in result_item.get("details", []):
            perturbed_answer = detail.get("perturbed_answer")
            if perturbed_answer:
                perturbed_answers.append(perturbed_answer)

    answer_scores = evaluator.evaluate(perturbed_answers, baseline_answer=final_answer)
    
    results.append(
        {
            "question": question_text,
            "final_answer": final_answer,
            "explanation": explanation,
            "metrics": metrics,
            "answer_scores": answer_scores,
            "documents": documents,
        }
    )

Perturbating 1 of 3
Perturbating 2 of 3
Perturbating 3 of 3
Perturbating 1 of 3
Perturbating 2 of 3
Perturbating 3 of 3
Perturbating 1 of 3
Perturbating 2 of 3
Perturbating 3 of 3
Perturbating 1 of 3
Perturbating 2 of 3
Perturbating 3 of 3
Perturbating 1 of 3
Perturbating 2 of 3
Perturbating 3 of 3
Perturbating 1 of 3
Perturbating 2 of 3
Perturbating 3 of 3
Perturbating 1 of 3
Perturbating 2 of 3
Perturbating 3 of 3
Perturbating 1 of 3
Perturbating 2 of 3
Perturbating 3 of 3
Perturbating 1 of 3
Perturbating 2 of 3
Perturbating 3 of 3
Perturbating 1 of 3
Perturbating 2 of 3
Perturbating 3 of 3


In [8]:
for i, res in enumerate(results):
    print(f"=== Result {i + 1} ===")
    print(res["question"])
    print(res["final_answer"])

    result = res["explanation"]
    metrics = res["metrics"]
    documents = res["documents"]

    print("Explain metrics:", metrics)
    print(RAGExExplainable.prettify(result))

    print(f"=== Result {i + 1} END ===")

=== Result 1 ===
Welche Autofarbe hat Tom?
Tom hat ein blaues Auto.
Explain metrics: {'duration_seconds': 161.24790545809083, 'steps': 7}
RAG-Ex Explanation Results

🎯 Interpretability (Feature Importance vs Ground Truth):
  • Jaccard Score: 1.0000
  • Top Importance: 1.0000
  • Intersection: 5 tokens
  • Union: 5 tokens

[CONTEXT] Token 0: "Max hat ein rotes Auto"
  Importance: 0.6825
  Per-Strategy Details:
    • leave_one_out:
      - Importance (raw): 0.4709
      - Similarity: 0.5291
      - NLI: ENTAILMENT (ent: 0.994) - Can explain: ✓
    • random_noise:
      - Importance (raw): 0.4709
      - Similarity: 0.5291
      - NLI: ENTAILMENT (ent: 0.996) - Can explain: ✓

[CONTEXT] Token 2: "Tom hat ein blaues Auto"
  Importance: 1.0000
  Per-Strategy Details:
    • leave_one_out:
      - Importance (raw): 0.6900
      - Similarity: 0.3100
      - NLI: NEUTRAL (ent: 0.001) - Can explain: ✗
    • random_noise:
      - Importance (raw): 0.0000
      - Similarity: 1.0000
      - NLI: EN